In [1]:
import random
import math
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
class TicTacToe:

    winning_combinations = [
      [0,1,2],
      [3,4,5],
      [6,7,8],
      [0,3,6],
      [1,4,7],
      [2,5,8],
      [0,4,8],
      [2,4,6]
    ]

    def __init__(self):
        self.data = []
        self.status = "Ongoing"
        self.board = np.zeros(9)
        self.winner = 0
        self.next_player = 1
        self.winning_combination = []

    def check_if_winner(self):
        for i in range(len(TicTacToe.winning_combinations)):
            if np.all( self.board[TicTacToe.winning_combinations[i]] == self.next_player):
                self.winner = self.next_player
                self.winning_combination = TicTacToe.winning_combinations[i]
                return True

        return False

    def switch_player(self):
        if self.next_player==1:
            self.next_player = 2
        elif self.next_player==2:
            self.next_player = 1
        else:
            print("Error!! Invalid player")
        return

    def next_moves(self):
        if self.status != 'Ongoing':
            return list()
        next_boards = list()
        for i in range(9):
            if self.board[i]==0:
                next_board = np.copy(self.board)
                next_board[i] = self.next_player
                next_boards.append(next_board)
        return next_boards

    def play(self,board):
        self.board = board
        #print(board, self.check_if_winner())
        if self.check_if_winner():
            #print("[Winner]:", self.winner)
            #print("combination: ", self.winning_combination)
            self.status = "Winner"
        else:
            self.switch_player()
            if np.count_nonzero(board==0)==0:
                self.status = "Draw"




In [3]:
ttt = TicTacToe()

while ttt.status=="Ongoing":

    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)
    print(np.reshape(ttt.board, (3,3)),"\n")
    if len(valid_moves)==0:
        break

print("[Winner]:",ttt.winner)

[[0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]] 

[[0. 0. 0.]
 [2. 0. 0.]
 [1. 0. 0.]] 

[[1. 0. 0.]
 [2. 0. 0.]
 [1. 0. 0.]] 

[[1. 0. 0.]
 [2. 0. 0.]
 [1. 2. 0.]] 

[[1. 0. 0.]
 [2. 0. 0.]
 [1. 2. 1.]] 

[[1. 0. 0.]
 [2. 2. 0.]
 [1. 2. 1.]] 

[[1. 0. 0.]
 [2. 2. 1.]
 [1. 2. 1.]] 

[[1. 0. 2.]
 [2. 2. 1.]
 [1. 2. 1.]] 

[[1. 1. 2.]
 [2. 2. 1.]
 [1. 2. 1.]] 

[Winner]: 0


In [4]:
winning_combination_tensor = tf.constant(TicTacToe.winning_combinations, dtype=tf.float32)

hidden_dimensions = 81

combinations = tf.constant(
    [
        [1,0,0,1,0,0,1,0,0],
        [0,1,0,0,1,0,0,1,0],
        [0,0,1,0,0,1,0,0,1],
        [1,1,1,0,0,0,0,0,0],
        [0,0,0,1,1,1,0,0,0],
        [0,0,0,0,0,0,1,1,1],
        [1,0,0,0,1,0,0,0,1],
        [0,0,1,0,1,0,1,0,0]
    ]
)

inputs = tf.constant([[0, 0, 0, 0, 0, 0, 0, 2, 1],[0, 1, 2, 1, 0, 0, 0, 0, 0]], dtype=tf.float32)
print(inputs)

x = tf.one_hot(tf.cast(inputs, tf.int32),3)
x = tf.keras.layers.Flatten()(x)
#x = tf.keras.layers.CategoryEncoding(num_tokens=3, output_mode='multi_hot')(inputs)
print(x)
x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=4, attention_axes=1)(x,x)
#x = tf.keras.layers.Flatten()(x)
#x = tf.keras.layers.Conv1D(81, 2, activation=tf.nn.softplus)(inputs)
print(x)
print(tf.shape(x))
x = tf.keras.layers.Dense(hidden_dimensions, activation=tf.nn.softplus)(x)
print(tf.shape(x))
x = tf.keras.layers.Dropout(0.3)(x)
#x = tf.concat([winning_x, flatten_inputs, x],axis=1)
x = tf.concat([inputs, x],axis=1)
print(tf.shape(x))
x = tf.keras.layers.Dense(9, activation=tf.nn.softplus)(x),
print(tf.shape(x))
x = tf.reshape(x, (2, 9))
zero = tf.constant(0, dtype=tf.float32)
mask = tf.equal(inputs, zero)
print(x)
x = tf.keras.layers.Softmax(axis=[0,1])(x, mask=mask)
print(x)

outputs = x


tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 2. 1.]
 [0. 1. 2. 1. 0. 0. 0. 0. 0.]], shape=(2, 9), dtype=float32)
tf.Tensor(
[[1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.
  0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
  1. 0. 0.]], shape=(2, 27), dtype=float32)
tf.Tensor(
[[ 0.00271109 -0.15537772  0.09266037 -0.13910519 -0.0113087   0.02255228
   0.0439573  -0.01894829  0.18276006  0.00273435 -0.03205568  0.02559961
   0.05553791 -0.11353954 -0.04800615 -0.05583434  0.09714416  0.19104251
  -0.13293439  0.06503929  0.11035983 -0.11477215  0.14982285  0.0362634
   0.0588077  -0.08179118  0.15680295]
 [ 0.08545551 -0.17458847 -0.04757813 -0.1058883  -0.1222957   0.05278289
   0.00349846 -0.01190934  0.12445813 -0.07797712 -0.12200741 -0.0016469
  -0.04403608 -0.07006781 -0.10982622 -0.08126004  0.12164955  0.2093898
  -0.08268096  0.11200728  0.08934669 -0.02442374  0.07190464  0.00091693
   0.03786714 -0.15216155  0.15417477]], sha

In [5]:
class MaskPlayed(tf.keras.layers.Layer):

    def __init__(self):
        super().__init__()
        self.trainable = True

    def call(self, inputs, x):
        zero = tf.constant(0, dtype=tf.float32)
        mask = tf.equal(inputs, zero)
        masked = tf.where(mask, x, tf.zeros_like(x))

        return masked


In [60]:
def create_Q():
    
    activation_fn = tf.nn.leaky_relu
    
    winning_combination_tensor = tf.constant(TicTacToe.winning_combinations, dtype=tf.float32)

    hidden_dimensions = 32
    depth = 4

    inputs = tf.keras.layers.Input((9))

    x = tf.one_hot(tf.cast(inputs, tf.int32),3)
    one_hot_inputs = tf.keras.layers.Flatten()(x)
    
    #x = tf.keras.layers.MultiHeadAttention(num_heads=120, key_dim=120, attention_axes=1)(one_hot_inputs,one_hot_inputs)
    x = tf.keras.layers.Dense(hidden_dimensions, activation=activation_fn)(one_hot_inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    for i in range(depth):
        x = tf.keras.layers.Dense(hidden_dimensions, activation=activation_fn)(x)
        x = tf.keras.layers.Dropout(0.2)(x)
    #x = tf.keras.layers.Dense(27, activation=activation_fn)(x)
    #x = tf.keras.layers.Dropout(0.3)(x)
    
    x = tf.concat([one_hot_inputs, x],axis=1)
    
    x = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
    #x = tf.reshape(x, (-1,9))
    #x = MaskPlayed()(inputs, x)

    outputs = x

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="tictactoe_model")

    return model


In [61]:
Q = create_Q()

In [62]:
output = Q( np.array([[0., 0., 0., 0., 0., 0., 0., 1., 2.]]) )
output

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7033988]], dtype=float32)>

In [63]:
Q.summary()

Model: "tictactoe_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_46 (InputLayer)          [(None, 9)]          0           []                               
                                                                                                  
 tf.cast_45 (TFOpLambda)        (None, 9)            0           ['input_46[0][0]']               
                                                                                                  
 tf.one_hot_45 (TFOpLambda)     (None, 9, 3)         0           ['tf.cast_45[0][0]']             
                                                                                                  
 flatten_46 (Flatten)           (None, 27)           0           ['tf.one_hot_45[0][0]']          
                                                                                    

In [64]:
tf.keras.utils.plot_model(Q, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


zero_weights = []
for arr in Qs[1].get_weights():
    zero_weights.append(np.zeros(arr.shape))
Qs[1].set_weights(zero_weights)
Qs[2].set_weights(zero_weights)

In [ ]:
BATCH_SIZE = 200

N = 1000
M = BATCH_SIZE
#N_replay = 3

player_model = {1:create_Q(), 2:create_Q()}

history = {
    'loss':[],
    'draw':[],
    'X':[],
    'O':[]
    }

epsilons = np.linspace(1.0, 0.3,N)
gamma = tf.constant(0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.Huber()

for n in range(N):
    
    epsilon = epsilons[n]

    winners = []

    played_boards = []
    board_rewards = {1:[], 2:[]}
    Q_j_plus_1 = {1:[], 2:[]}

    ttt_games = [TicTacToe() for i in range(M)]

    # loop over the maximum length of the game
    for turn in range(9):

        for g in range(len(ttt_games)):

            next_boards = ttt_games[g].next_moves()
            possible_boards = tf.concat([ ttt_games[i].board[np.newaxis,...] for i in range(len(ttt_games))], axis=0)
    
        board_scores = (player_model[ttt_games[0].next_player](current_boards))
        #print("[scores]", board_scores)

        best_scores = tf.math.argmax(board_scores,axis=1)
        #print("[best score]", best_scores)

        # loop over all the single games
        for i in range(M):
            
            game = ttt_games[i]
            if game.status=="Ongoing":
                
                next_boards = game.next_moves()
                if random.random()>epsilon:
                    current_player = game.next_player
                    forward_boards = tf.concat([ b[np.newaxis,...] for b in next_boards], axis=0)
                    board_scores = (player_model[current_player](forward_boards))
                    #print("[scores]", board_scores)
                    best_score = tf.math.argmax(board_scores,axis=0)
                    #print("[best score]", best_score)
                    draw = np.random.choice(best_score.numpy())
                    #print(draw)
                else:
                    draw = np.random.choice(range(len(next_boards)))
                
                next_board = next_boards[draw]
                game.play(next_board)
      
            for p in board_rewards:
                if game.status=="Ongoing":
                    board_rewards[p] += [0.0]

                elif game.status == "Winner":
                    if p==game.winner:
                        board_rewards[p].append(1.0)
                    else:
                        board_rewards[p].append(0.0)

                elif game.status == "Draw":
                    board_rewards[p] += [0.0]

                else:
                    print("Something is wrong")
        
        current_boards = tf.concat([ ttt_games[i].board[np.newaxis,...] for i in range(len(ttt_games))], axis=0)
        played_boards.append(current_boards)
        #print("[boards]", current_boards)

        for i in range(len(ttt_games)):

            next_boards = ttt_games[i].next_moves()
            
            for p in Q_j_plus_1:
                
                if len(next_boards)>0:

                    forward_boards = tf.concat([ b[np.newaxis,...] for b in next_boards], axis=0)
                    forward_scores = player_model[p](forward_boards)
                    qs_plus_1 = tf.reduce_max(forward_scores).numpy()
                    Q_j_plus_1[p].append(qs_plus_1)

                else:
                    Q_j_plus_1[p].append(0.0)
        
        #for p in board_rewards:
        #    print("[Q+1]",p, Q_j_plus_1[p][-BATCH_SIZE:])
        #    print("[R]",p, board_rewards[p][-BATCH_SIZE:])

    played_boards = tf.concat(played_boards, axis=0)
    for p in Q_j_plus_1:
        Q_j_plus_1[p] = tf.concat(Q_j_plus_1[p], axis=0)

    for game in ttt_games:
        winners.append(game.winner)

  
    total_loss = 0

    for p in player_model:
        
        for retrain in range(9):

            training_replay = range(played_boards.shape[0])[(8-retrain)*BATCH_SIZE:(9-retrain)*BATCH_SIZE]

            training_boards = tf.concat( [played_boards[i,...][np.newaxis,...] for i in training_replay], axis=0 )

            rewards = [ board_rewards[p][i] for i in training_replay ]

            q_plus_1 = [ Q_j_plus_1[p][i] for i in training_replay]

            #print(tf.shape(training_boards), tf.shape(rewards), tf.shape(q_plus_1))

            with tf.GradientTape() as tape:
                y = player_model[p](training_boards, training=True)
                #y = tf.math.reduce_max(y,axis=1)
                loss_values = loss_fn(rewards + gamma*q_plus_1, y)
                #print(y,loss_values)

            grads = tape.gradient(loss_values, player_model[p].trainable_variables)
            optimizer.apply_gradients(zip(grads, player_model[p].trainable_variables))
            total_loss += tf.reduce_mean(loss_values)

    print(n+1, round(epsilon,2), [(x,winners.count(x)/M) for x in set(winners)], total_loss.numpy())
    history['loss'].append(total_loss)
    history['draw'].append(winners.count(0)/M)
    history['X'].append(winners.count(1)/M)
    history['O'].append(winners.count(2)/M)
    #break

1 1.0 [(0, 0.14), (1, 0.58), (2, 0.28)] 0.6428904
2 1.0 [(0, 0.105), (1, 0.545), (2, 0.35)] 0.63130444
3 1.0 [(0, 0.115), (1, 0.585), (2, 0.3)] 0.54702777
4 1.0 [(0, 0.125), (1, 0.59), (2, 0.285)] 0.503231
5 1.0 [(0, 0.14), (1, 0.58), (2, 0.28)] 0.51764584
6 1.0 [(0, 0.13), (1, 0.595), (2, 0.275)] 0.49546084
7 1.0 [(0, 0.095), (1, 0.61), (2, 0.295)] 0.53959244
8 1.0 [(0, 0.14), (1, 0.58), (2, 0.28)] 0.48963594
9 0.99 [(0, 0.095), (1, 0.57), (2, 0.335)] 0.46613857
10 0.99 [(0, 0.12), (1, 0.615), (2, 0.265)] 0.4876807
11 0.99 [(0, 0.095), (1, 0.61), (2, 0.295)] 0.45683342
12 0.99 [(0, 0.145), (1, 0.565), (2, 0.29)] 0.50847167
13 0.99 [(0, 0.15), (1, 0.545), (2, 0.305)] 0.45268285
14 0.99 [(0, 0.11), (1, 0.62), (2, 0.27)] 0.4410053
15 0.99 [(0, 0.115), (1, 0.505), (2, 0.38)] 0.45764586
16 0.99 [(0, 0.115), (1, 0.59), (2, 0.295)] 0.4572026
17 0.99 [(0, 0.14), (1, 0.57), (2, 0.29)] 0.44557625
18 0.99 [(0, 0.14), (1, 0.55), (2, 0.31)] 0.44245672
19 0.99 [(0, 0.135), (1, 0.57), (2, 0.295)] 0.

153 0.89 [(0, 0.095), (1, 0.56), (2, 0.345)] 0.27225026
154 0.89 [(0, 0.15), (1, 0.585), (2, 0.265)] 0.24622217
155 0.89 [(0, 0.11), (1, 0.57), (2, 0.32)] 0.27224198
156 0.89 [(0, 0.105), (1, 0.575), (2, 0.32)] 0.27160588
157 0.89 [(0, 0.12), (1, 0.555), (2, 0.325)] 0.2894264
158 0.89 [(0, 0.1), (1, 0.59), (2, 0.31)] 0.26016238
159 0.89 [(0, 0.075), (1, 0.605), (2, 0.32)] 0.25672203
160 0.89 [(0, 0.06), (1, 0.62), (2, 0.32)] 0.26944432
161 0.89 [(0, 0.13), (1, 0.6), (2, 0.27)] 0.2903444
162 0.89 [(0, 0.11), (1, 0.63), (2, 0.26)] 0.2695422
163 0.89 [(0, 0.06), (1, 0.615), (2, 0.325)] 0.27132612
164 0.89 [(0, 0.09), (1, 0.58), (2, 0.33)] 0.28087506
165 0.89 [(0, 0.11), (1, 0.62), (2, 0.27)] 0.26201394
166 0.88 [(0, 0.11), (1, 0.6), (2, 0.29)] 0.27005178
167 0.88 [(0, 0.095), (1, 0.59), (2, 0.315)] 0.25945002
168 0.88 [(0, 0.09), (1, 0.605), (2, 0.305)] 0.26685357
169 0.88 [(0, 0.115), (1, 0.61), (2, 0.275)] 0.25568017
170 0.88 [(0, 0.075), (1, 0.595), (2, 0.33)] 0.28399494
171 0.88 [(0, 

302 0.79 [(0, 0.075), (1, 0.675), (2, 0.25)] 0.23750238
303 0.79 [(0, 0.075), (1, 0.635), (2, 0.29)] 0.22879462
304 0.79 [(0, 0.055), (1, 0.605), (2, 0.34)] 0.228979
305 0.79 [(0, 0.08), (1, 0.655), (2, 0.265)] 0.20805252
306 0.79 [(0, 0.02), (1, 0.635), (2, 0.345)] 0.21384555
307 0.79 [(0, 0.07), (1, 0.655), (2, 0.275)] 0.23077792
308 0.78 [(0, 0.065), (1, 0.665), (2, 0.27)] 0.23163989
309 0.78 [(0, 0.065), (1, 0.625), (2, 0.31)] 0.22577137
310 0.78 [(0, 0.075), (1, 0.64), (2, 0.285)] 0.23375902
311 0.78 [(0, 0.095), (1, 0.65), (2, 0.255)] 0.2030694
312 0.78 [(0, 0.08), (1, 0.605), (2, 0.315)] 0.2378335
313 0.78 [(0, 0.07), (1, 0.645), (2, 0.285)] 0.2115644
314 0.78 [(0, 0.07), (1, 0.64), (2, 0.29)] 0.22391747
315 0.78 [(0, 0.04), (1, 0.615), (2, 0.345)] 0.23329712
316 0.78 [(0, 0.095), (1, 0.65), (2, 0.255)] 0.21714234
317 0.78 [(0, 0.055), (1, 0.7), (2, 0.245)] 0.2125822
318 0.78 [(0, 0.045), (1, 0.65), (2, 0.305)] 0.21407102
319 0.78 [(0, 0.075), (1, 0.6), (2, 0.325)] 0.23376682
32

451 0.68 [(0, 0.085), (1, 0.685), (2, 0.23)] 0.1884824
452 0.68 [(0, 0.045), (1, 0.595), (2, 0.36)] 0.21307641
453 0.68 [(0, 0.08), (1, 0.635), (2, 0.285)] 0.23303089
454 0.68 [(0, 0.04), (1, 0.635), (2, 0.325)] 0.20322484
455 0.68 [(0, 0.06), (1, 0.56), (2, 0.38)] 0.23000963
456 0.68 [(0, 0.075), (1, 0.57), (2, 0.355)] 0.20836236
457 0.68 [(0, 0.07), (1, 0.61), (2, 0.32)] 0.21294193
458 0.68 [(0, 0.05), (1, 0.575), (2, 0.375)] 0.24042258
459 0.68 [(0, 0.09), (1, 0.6), (2, 0.31)] 0.23218608
460 0.68 [(0, 0.08), (1, 0.59), (2, 0.33)] 0.24285342
461 0.68 [(0, 0.065), (1, 0.6), (2, 0.335)] 0.23617822
462 0.68 [(0, 0.05), (1, 0.66), (2, 0.29)] 0.21184042
463 0.68 [(0, 0.035), (1, 0.625), (2, 0.34)] 0.23289144
464 0.68 [(0, 0.065), (1, 0.54), (2, 0.395)] 0.23369916
465 0.67 [(0, 0.065), (1, 0.555), (2, 0.38)] 0.22462273
466 0.67 [(0, 0.06), (1, 0.65), (2, 0.29)] 0.23046309
467 0.67 [(0, 0.035), (1, 0.615), (2, 0.35)] 0.22952394
468 0.67 [(0, 0.075), (1, 0.635), (2, 0.29)] 0.20641541
469 0.6

600 0.58 [(0, 0.015), (1, 0.63), (2, 0.355)] 0.15881139
601 0.58 [(0, 0.025), (1, 0.65), (2, 0.325)] 0.17449233
602 0.58 [(0, 0.03), (1, 0.695), (2, 0.275)] 0.2058767
603 0.58 [(0, 0.04), (1, 0.715), (2, 0.245)] 0.20890233
604 0.58 [(0, 0.02), (1, 0.64), (2, 0.34)] 0.20035906
605 0.58 [(0, 0.025), (1, 0.695), (2, 0.28)] 0.2162367
606 0.58 [(0, 0.03), (1, 0.63), (2, 0.34)] 0.24236958
607 0.58 [(0, 0.02), (1, 0.585), (2, 0.395)] 0.1954643
608 0.57 [(0, 0.065), (1, 0.605), (2, 0.33)] 0.21125361
609 0.57 [(0, 0.025), (1, 0.68), (2, 0.295)] 0.2082669
610 0.57 [(0, 0.015), (1, 0.675), (2, 0.31)] 0.18524599
611 0.57 [(0, 0.035), (1, 0.665), (2, 0.3)] 0.17487626
612 0.57 [(0, 0.04), (1, 0.62), (2, 0.34)] 0.17925985
613 0.57 [(0, 0.025), (1, 0.65), (2, 0.325)] 0.17831127
614 0.57 [(0, 0.03), (1, 0.63), (2, 0.34)] 0.18032217
615 0.57 [(0, 0.035), (1, 0.71), (2, 0.255)] 0.22490099
616 0.57 [(0, 0.045), (1, 0.725), (2, 0.23)] 0.22596617
617 0.57 [(0, 0.03), (1, 0.685), (2, 0.285)] 0.31360656
618 0

749 0.48 [(0, 0.025), (1, 0.655), (2, 0.32)] 0.15730612
750 0.48 [(0, 0.02), (1, 0.695), (2, 0.285)] 0.15456043
751 0.47 [(0, 0.02), (1, 0.76), (2, 0.22)] 0.16750704
752 0.47 [(0, 0.02), (1, 0.74), (2, 0.24)] 0.18020286
753 0.47 [(0, 0.03), (1, 0.67), (2, 0.3)] 0.20273872
754 0.47 [(0, 0.025), (1, 0.74), (2, 0.235)] 0.15831387
755 0.47 [(0, 0.02), (1, 0.69), (2, 0.29)] 0.20092283
756 0.47 [(0, 0.01), (1, 0.655), (2, 0.335)] 0.1995341
757 0.47 [(0, 0.015), (1, 0.65), (2, 0.335)] 0.16318448
758 0.47 [(0, 0.01), (1, 0.72), (2, 0.27)] 0.17985624
759 0.47 [(0, 0.01), (1, 0.675), (2, 0.315)] 0.19607078
760 0.47 [(0, 0.02), (1, 0.72), (2, 0.26)] 0.17074181
761 0.47 [(0, 0.03), (1, 0.685), (2, 0.285)] 0.1755388
762 0.47 [(0, 0.05), (1, 0.64), (2, 0.31)] 0.1919841
763 0.47 [(0, 0.03), (1, 0.68), (2, 0.29)] 0.19465853
764 0.47 [(0, 0.02), (1, 0.59), (2, 0.39)] 0.19598292
765 0.46 [(0, 0.04), (1, 0.675), (2, 0.285)] 0.22249341
766 0.46 [(0, 0.01), (1, 0.625), (2, 0.365)] 0.17398082
767 0.46 [(0, 

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(history['loss'])
ax[1].set( ylabel='loss')
ax[1].plot(history['draw'], label='draw')
ax[1].plot(history['X'], label='X')
ax[1].plot(history['O'], label='O')
ax[1].set( ylim=[0,1], xlabel='epoch', ylabel='probability')
ax[1].legend()

In [ ]:
ttt = TicTacToe()

valid_moves = ttt.next_moves()
new_board = random.choice(valid_moves)
ttt.play(new_board)
print(np.reshape(ttt.board,(3,3)),"\n")

while ttt.status=="Ongoing":
    current_board = ttt.board
    valid_moves = ttt.next_moves()
    current_player = ttt.next_player
    if len(valid_moves)==0:
        break
    scores = player_model[current_player].predict( tf.concat( [b[np.newaxis,...] for b in valid_moves], axis=0) )
    scores = scores.flatten()
    print(scores)
    max_idx = np.argmax(scores)
    next_board = valid_moves[max_idx]
    ttt.play(next_board)
    print(np.reshape(ttt.board,(3,3)),"\n")

print("[Winner]:",ttt.winner)

In [ ]:
winners = []
for i in range(100):
    ttt = TicTacToe()
    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)

    while ttt.status=="Ongoing":
        current_board = ttt.board
        valid_moves = ttt.next_moves()
        current_player = ttt.next_player
        if len(valid_moves)==0:
            break
        scores = player_model[current_player].predict( tf.concat( [b[np.newaxis,...] for b in valid_moves], axis=0) )
        scores = scores.flatten()
        #print(scores)
        max_idx = np.argmax(scores)
        next_board = valid_moves[max_idx]
        ttt.play(next_board)
        #print(ttt.board,"\n")

    winners.append(ttt.winner)

In [ ]:
winners_against_random = []
for i in range(100):
    ttt = TicTacToe()
    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)

    while ttt.status=="Ongoing":
        current_board = ttt.board
        valid_moves = ttt.next_moves()
        if len(valid_moves)==0:
            break
        current_player = ttt.next_player
        if current_player==1:
            scores = player_model[current_player].predict(tf.concat( [b[np.newaxis,...] for b in valid_moves], axis=0))
            scores = scores.flatten()
            max_idx = np.argmax(scores)
            next_board = valid_moves[max_idx]
        else:
            next_board = random.choice(valid_moves)
        ttt.play(next_board)

    winners_against_random.append(ttt.winner)

In [ ]:
random_winners = []
for i in range(100):
    ttt = TicTacToe()
    while ttt.status=="Ongoing":
        valid_moves = ttt.next_moves()
        if len(valid_moves)==0:
            break
        new_board = random.choice(valid_moves)
        ttt.play(new_board)
    ttt.board
    random_winners.append(ttt.winner)

In [ ]:
df_ai = pd.Series({x:winners.count(x) for x in set(winners)}, name='AI').to_frame()
df_ai_against_random = pd.Series({x:winners_against_random.count(x) for x in set(winners_against_random)}, name='AI against random').to_frame()
df_random = pd.Series({x:random_winners.count(x) for x in set(random_winners)}, name='random').to_frame()

df = pd.concat([df_random, df_ai, df_ai_against_random], axis=1)

df = df/df.sum(axis=0)

df.index = df.index.map({0:'Draw',1:"X",2:"O"})

df.T.plot(kind='bar', stacked=True)

In [20]:
player_model[1].save('saved_model/TTT_model_player_1')
player_model[2].save('saved_model/TTT_model_player_2')
!bash conversion.sh

INFO:tensorflow:Assets written to: saved_model/TTT_model_player_1/assets
INFO:tensorflow:Assets written to: saved_model/TTT_model_player_2/assets
